install packagees

In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer, LabelEncoder
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
)
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import matplotlib.pyplot as plt


prepare/process data
 - normalize numerical col
 - create one hot encoding
 - create feature matric

In [14]:
data = pd.read_csv("cleaned_data.csv")
user_data = pd.read_csv("user_data.csv")
#print(data.head())

# Convert 'startYear' to numeric if it's not already
data['startYear'] = pd.to_numeric(data['startYear'], errors='coerce')

# Handle missing values in numerical columns by filling with the mean or median
numerical_cols = ['startYear', 'averageRating', 'numVotes']
for col in numerical_cols:
    if data[col].isnull().sum() > 0:
        data[col].fillna(data[col].median(), inplace=True)
        #print(f"Filled missing values in {col} with median.")

# For categorical columns, fill missing values with a placeholder or mode
categorical_cols = ['titleType', 'genres', 'directorNames', 'writerNames', 'isAdult']
for col in categorical_cols:
    data[col].fillna('Unknown', inplace=True)
    #print(f"Filled missing values in {col} with 'Unknown'.")

data = data[data['startYear'] > 2000]
print(data.head())
#print(user_data.head())

          tconst titleType                                       primaryTitle  \
14035  tt0035423     movie                                     Kate & Leopold   
33106  tt0062336     movie  The Tango of the Widower and Its Distorting Mi...   
36801  tt0067758     movie                            Simón, contamos contigo   
37664  tt0069049     movie                         The Other Side of the Wind   
38671  tt0070596     movie                                  Socialist Realism   

       isAdult  startYear                            genres  \
14035        0       2001  ['Comedy', 'Fantasy', 'Romance']   
33106        0       2020                         ['Drama']   
36801        0       2015               ['Comedy', 'Drama']   
37664        0       2018                         ['Drama']   
38671        0       2023                         ['Drama']   

                             directorNames  \
14035                    ['James Mangold']   
33106  ['Raúl Ruiz', ' Valeria Sarmiento']

/var/folders/1y/4sm5vl2d7_xdd0jwk0tgnhn00000gn/T/ipykernel_12161/1304589896.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna('Unknown', inplace=True)


In [15]:
# movie data

# Initialize the scaler
scaler = MinMaxScaler()

# Select numerical columns
numerical_cols = ['startYear', 'averageRating', 'numVotes']

# Fit and transform the data
data = data.copy()
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

print("Numerical columns normalized successfully!")
data[numerical_cols].head()

# One-Hot Encode 'titleType' and 'isAdult'
data = pd.get_dummies(data, columns=['titleType', 'isAdult'], prefix=['titleType', 'isAdult'])

# 0 and 1 assignment for titleType_movie, isAdult_0, isAdult_1
data[["titleType_movie", "isAdult_0", "isAdult_1"]] = data[["titleType_movie", "isAdult_0", "isAdult_1"]].astype(int)

# genre multi-encoding
data['genre_list'] = data['genres'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# mlb encoder
multi_label_encoder = MultiLabelBinarizer()
genres_encoded = multi_label_encoder.fit_transform(data['genre_list'])
genres_encoded_df = pd.DataFrame(genres_encoded, columns=multi_label_encoder.classes_)
data = pd.concat([data, genres_encoded_df], axis=1)


#print(user_data.head())

# director label encoding
movie_encoder = LabelEncoder()

data['directorNames'] = movie_encoder.fit_transform(data['directorNames'])

# writer label encoding
data['writerNames'] = movie_encoder.fit_transform(data['writerNames'])

# movie_id using label encoding
data["primaryTitle"] = movie_encoder.fit_transform(data["primaryTitle"])


data.head()
#print(data.columns)
#user_data.head()





Numerical columns normalized successfully!


,tconst,primaryTitle,startYear,genres,directorNames,writerNames,averageRating,numVotes,titleType_movie,isAdult_0,...,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western
14035,tt0035423,71618,0.000000,"['Comedy', 'Fantasy', 'Romance']",41219,110297,0.600000,0.030765,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33106,tt0062336,146903,0.826087,['Drama'],79534,96010,0.611111,0.000064,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36801,tt0067758,120996,0.608696,"['Comedy', 'Drama']",78880,59871,0.355556,0.000014,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37664,tt0069049,142964,0.739130,['Drama'],72832,88176,0.633333,0.002780,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
38671,tt0070596,122641,0.956522,['Drama'],79534,19735,0.711111,0.000023,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [16]:
#user_data

user_numerical_cols = ['UserRating']

user_data[user_numerical_cols] = scaler.fit_transform(user_data[user_numerical_cols])

user_data['user_genre_list'] = user_data['FavoriteGenres'].apply(lambda x: eval(x) if isinstance(x, str) else x)
user_genres_encoded = multi_label_encoder.fit_transform(user_data['user_genre_list'])
user_genres_encoded_df = pd.DataFrame(user_genres_encoded, columns=multi_label_encoder.classes_)
user_data = pd.concat([user_data, user_genres_encoded_df], axis=1)

user_encoder = LabelEncoder()

# director label encoding
user_data['FavoriteDirectors'] = user_encoder.fit_transform(user_data['FavoriteDirectors'])

# writer label encoding
user_data['FavoriteActors'] = user_encoder.fit_transform(user_data['FavoriteActors'])

# movie_id label encoding
user_data["primaryTitle"] = user_encoder.fit_transform(user_data["primaryTitle"])

# user_id label ecoding
user_data["UserID"] = user_encoder.fit_transform(user_data["UserID"])


user_data.head()
#print(user_data.columns)

,UserID,tconst,primaryTitle,UserRating,FavoriteGenres,FavoriteActors,FavoriteDirectors,user_genre_list,Action,Adult,...,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western
0,0,tt0000009,149138,0.000000,['Romance'],149,19868,[Romance],0,0,...,0,0,0,1,0,0,0,0,0,0
1,111111,tt0000147,224595,0.444444,"['News', 'Documentary', 'Sport']",49,46569,"[News, Documentary, Sport]",0,0,...,0,1,0,0,0,1,0,0,0,0
2,222222,tt0000502,35350,0.888889,[],299,105924,[],0,0,...,0,0,0,0,0,0,0,0,0,0
3,252417,tt0000574,247271,0.777778,['Action'],249,33864,[Action],1,0,...,0,0,0,0,0,0,0,0,0,0
4,263528,tt0000591,242575,0.888889,['Drama'],144,89998,[Drama],0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
genre_list = [col for col in data.columns if col not in ['tconst', 'primaryTitle', 'startYear', 'genres', 'directorNames',
       'writerNames', 'averageRating', 'numVotes', 'titleType_movie',
       'isAdult_0', 'isAdult_1', 'genre_list']]

movie_features = pd.concat([data[['tconst', 'averageRating', 'numVotes']]],axis=1)

movie_features['genres_list'] = data[genre_list].apply(
    lambda genres: [1 if genres[genre] == 1 else 0 for genre in genre_list], axis=1  # Access genre columns by name
)

# formats user features in an array
user_features = pd.concat([user_data[['tconst', 'UserID', 'UserRating']]],axis=1)

# merges user and movie data into an array
merged_data = pd.merge(user_features, movie_features, on='tconst', how='inner')

#creates a dictionary for tconst to movie
tconst_to_title = data.set_index('tconst')['primaryTitle'].to_dict()

merged_data.head()
#print(tconst_to_title)


,tconst,UserID,UserRating,averageRating,numVotes,genres_list
0,tt0035423,44847,0.444444,0.600000,0.030765,"[1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,tt0062336,244761,0.777778,0.611111,0.000064,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
2,tt0067758,248866,0.222222,0.355556,0.000014,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,tt0069049,249824,0.888889,0.633333,0.002780,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
4,tt0070596,250943,0.555556,0.711111,0.000023,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, ..."


In [18]:
x_input = []

for index, row in merged_data.iterrows():
    x_input.append(list(row['genres_list']) + [row['averageRating'], row['numVotes'], row['UserRating']])

print(len(x_input[0]))

29


create vae model design
 - encoder
 - decoder
 - define loss function

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions

class Encoder(nn.Module):

    def __init__( self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, hidden_dim5, latent_dim ):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4 = nn.Linear(hidden_dim3, hidden_dim4)
        self.fc5 = nn.Linear(hidden_dim4, hidden_dim5)
        self.fc_mu = nn.Linear(hidden_dim5, latent_dim)      # Mean of latent distribution
        self.fc_log_var = nn.Linear(hidden_dim5, latent_dim) # Log variance of latent distribution

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        mu = self.fc_mu(x)
        log_var = self.fc_log_var(x)
        return mu, log_var  # Return mean and log variance

class Decoder(nn.Module):

    def __init__( self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, hidden_dim5, latent_dim ):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, hidden_dim5)
        self.fc2 = nn.Linear(hidden_dim5, hidden_dim4)
        self.fc3 = nn.Linear(hidden_dim4, hidden_dim3)
        self.fc4 = nn.Linear(hidden_dim3, hidden_dim2)
        self.fc5 = nn.Linear(hidden_dim2, hidden_dim1)
        self.fc_out = nn.Linear(hidden_dim1, input_dim)

    def forward(self, z):
        z = F.relu(self.fc1(z))
        z = F.relu(self.fc2(z))
        z = F.relu(self.fc3(z))
        z = F.relu(self.fc4(z))
        z = F.relu(self.fc5(z))
        # might want to change .softmax to .sigmoid
        return F.sigmoid(self.fc_out(z))  # Return class probabilities

class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, hidden_dim5, latent_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, hidden_dim5, latent_dim)
        self.decoder = Decoder(input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, hidden_dim5, latent_dim)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)  # Standard deviation
        epsilon = torch.randn_like(std)  # Random noise
        return mu + epsilon * std  # Reparameterization trick

    def forward(self, x):
        mu, log_var = self.encoder(x)  # Encode input to latent distribution
        z = self.reparameterize(mu, log_var)  # Sample latent variable
        x_reconstructed = self.decoder(z)  # Decode back to input space
        return x_reconstructed, mu, log_var  # Return reconstructed data, mean, and log variance


def loss_function(recon_x, x, mu, logvar, binary_mask, continuous_mask, beta=1.0):
    """
    Loss function for VAE with mixed binary and continuous data.

    recon_x: Reconstructed data (output of the decoder)
    x: Original input data
    mu: Mean of the latent distribution (output of the encoder)
    logvar: Log variance of the latent distribution (output of the encoder)
    binary_mask: A mask indicating which features are binary (True/False)
    continuous_mask: A mask indicating which features are continuous (True/False)
    beta: Scaling factor for the KL divergence term (optional)
    """

    # Binary Cross-Entropy for binary features
    binary_x = x[:, binary_mask]  # Select binary features
    recon_binary_x = recon_x[:, binary_mask]  # Reconstructed binary features
    binary_recon_loss = F.binary_cross_entropy(recon_binary_x, binary_x, reduction='sum')

    # Mean Squared Error for continuous features
    continuous_x = x[:, continuous_mask]  # Select continuous features
    recon_continuous_x = recon_x[:, continuous_mask]  # Reconstructed continuous features
    continuous_recon_loss = F.mse_loss(recon_continuous_x, continuous_x, reduction='sum')

    # Combine the reconstruction losses (you can scale continuous loss if necessary)
    total_recon_loss = binary_recon_loss + beta * continuous_recon_loss

    # KL Divergence Loss
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Total Loss = Reconstruction Loss + KL Divergence
    total_loss = total_recon_loss + kl_loss
    return total_loss

Training VAE
 - train model
  - forward pass
  - compute loss
  - backpropogation
  - run for # of epochs

In [20]:
# Hyperparameters
# try adding hidden layers
HIDDEN_DIM1 = 512
HIDDEN_DIM2 = 256
HIDDEN_DIM3 = 128
HIDDEN_DIM4 = 64
HIDDEN_DIM5 = 32
LATENT_DIM = 30
BIN_MASK = torch.tensor([True, True, True, True, True, True, True, True, True, True,
                        True, True, True, True, True, True, True, True, True, True,
                        True, True, True, True, True, True, False, False, False])
CONT_MASK = torch.tensor([False, False, False, False, False, False, False, False, False, False,
                          False, False, False, False, False, False, False, False, False, False,
                          False, False, False, False, False, False, True, True, True])

# Prepare data for training
merged_data_tensor = torch.tensor(x_input, dtype=torch.float32)

# Split data:
temp_dataset = TensorDataset(merged_data_tensor)
train_size = int(0.8 * len(temp_dataset))
val_size = len(temp_dataset) - train_size

train_dataset, val_dataset = random_split(temp_dataset, [train_size, val_size])

# hyperparameter tuning, change around the batch_size
# grid search, we can change around batch_size = {8,16,32,64}
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size =64, shuffle=False)

# Model setup
input_dim = merged_data_tensor.shape[1]
vae = VAE(input_dim, HIDDEN_DIM1, HIDDEN_DIM2, HIDDEN_DIM3, HIDDEN_DIM4, HIDDEN_DIM5, LATENT_DIM)

# changed learning rate to 1e-3 => 1e-5
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

# Training loop
# temporarily change EPOCHS=1
EPOCHS = 50
vae.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch in train_loader:
        batch = batch[0]  # Unpack the TensorDataset
        optimizer.zero_grad()

        # Forward pass
        recon_batch, mu, logvar = vae(batch)
        loss = loss_function(recon_batch, batch, mu, logvar, BIN_MASK, CONT_MASK)

        # Backward pass
        loss.backward()
        total_loss += loss.item()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader.dataset):.4f}")

print("VAE training complete!")

Epoch 1, Loss: 2.3743
Epoch 2, Loss: 2.2183
Epoch 3, Loss: 2.1809
Epoch 4, Loss: 2.1668
Epoch 5, Loss: 2.1572
Epoch 6, Loss: 2.1528
Epoch 7, Loss: 2.1542
Epoch 8, Loss: 2.1497
Epoch 9, Loss: 2.1464
Epoch 10, Loss: 2.1501
Epoch 11, Loss: 2.1466
Epoch 12, Loss: 2.1461
Epoch 13, Loss: 2.1439
Epoch 14, Loss: 2.1428
Epoch 15, Loss: 2.1434
Epoch 16, Loss: 2.1440
Epoch 17, Loss: 2.1422
Epoch 18, Loss: 2.1409
Epoch 19, Loss: 2.1405
Epoch 20, Loss: 2.1407
Epoch 21, Loss: 2.1380
Epoch 22, Loss: 2.1402
Epoch 23, Loss: 2.1365
Epoch 24, Loss: 2.1369
Epoch 25, Loss: 2.1378
Epoch 26, Loss: 2.1389
Epoch 27, Loss: 2.1383
Epoch 28, Loss: 2.1399
Epoch 29, Loss: 2.1351
Epoch 30, Loss: 2.1377
Epoch 31, Loss: 2.1359
Epoch 32, Loss: 2.1377
Epoch 33, Loss: 2.1342
Epoch 34, Loss: 2.1355
Epoch 35, Loss: 2.1356
Epoch 36, Loss: 2.1359
Epoch 37, Loss: 2.1369
Epoch 38, Loss: 2.1352
Epoch 39, Loss: 2.1367
Epoch 40, Loss: 2.1364
Epoch 41, Loss: 2.1311
Epoch 42, Loss: 2.1337
Epoch 43, Loss: 2.1341
Epoch 44, Loss: 2.13

Generate Latent Representations
- evaluate user preferences
- compare with other movies

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

def get_user_embeddings(user_rating, movie_features, vae_model):
    vae_model.eval()
    with torch.no_grad():
        input_vector = torch.tensor(movie_features + [user_rating], dtype=torch.float32)
        _, mu, _ = vae_model(input_vector)
    return mu.numpy()

def reccomend_movies(user_rating, movie_features, movie_embeddings, data, top_n=5):
    user_embedding = get_user_embeddings(user_rating, movie_features, vae)

    # added a reshape
    user_embedding = user_embedding.reshape(-1, 1)

    # code crashes here
    # print("User embeddings is: ", user_embedding)
    similarity_scores = cosine_similarity(user_embedding, movie_embeddings)

    top_indices = np.argsort(similarity_scores.flatten())[::-1][:top_n]

    recommended_movies = data.iloc[top_indices]

    recommended_movies['similarity_score'] = similarity_scores.flatten()[top_indices]

    return recommended_movies[['tconst', 'primaryTitle', 'averageRating', 'similarity_score']]


Content-Based Recomendations
 - Compute Similarities
 - generate reccomendations


In [25]:
movie_emeddings = []

vae.eval()
with torch.no_grad():
    for index, row in movie_features.iterrows():
        movie_input = torch.tensor(row['genres_list'] + [row['averageRating'], row['numVotes'], 0.5], dtype=torch.float32)
        # _, mu, _ => mu, logvar
        mu, _= vae.encoder(movie_input)
        movie_emeddings.append(mu.numpy())

# try changing NaN to 0
movie_emeddings = pd.DataFrame(movie_emeddings)
movie_emeddings = movie_emeddings.fillna(0)

user_rating = 4.5
movie_example_features = [0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0.5,
 6.057523510812793e-06]
movie_emeddings = np.vstack(movie_emeddings)
print("These are the movie embeddings:\n ", movie_emeddings)
# Error occurs here

These are the movie embeddings:
  [[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [13]
 [14]
 [15]
 [16]
 [17]
 [18]
 [19]
 [20]
 [21]
 [22]
 [23]
 [24]
 [25]
 [26]
 [27]
 [28]
 [29]]


Testing and Validations
 - Evaluate Reccomendations
 - Fine Tune Model

In [26]:
top_movies = reccomend_movies(user_rating, movie_example_features, movie_emeddings, data, top_n=5)
top_movies['primaryTitle'] = movie_encoder.inverse_transform(top_movies['primaryTitle'])
print(top_movies)

          tconst                        primaryTitle  averageRating  \
77718  tt0159369  Cooper and Hemingway: The True Gen       0.711111   
95540  tt0225828                              G.O.D.       0.322222   
81038  tt0172156                         Bad Boys II       0.622222   
81008  tt0171928                A Samba for Sherlock       0.622222   
80868  tt0171436      The Kidnapping of Chris Burden       0.711111   

       similarity_score  
77718               1.0  
95540               1.0  
81038               1.0  
81008               1.0  
80868               1.0  


/var/folders/1y/4sm5vl2d7_xdd0jwk0tgnhn00000gn/T/ipykernel_12161/2755284885.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_movies['similarity_score'] = similarity_scores.flatten()[top_indices]


Loss Compared to Validation Data

In [27]:
total_loss = 0

vae.eval()

with torch.no_grad():  # Disable gradient tracking since we're only evaluating
    for batch in val_loader:
        batch = batch[0]  # Unpack the TensorDataset (assuming only one tensor, batch[0] is the data)

        # Forward pass
        recon_batch, mu, logvar = vae(batch)

        # Calculate the loss (use your custom loss function, and include masks if necessary)
        loss = loss_function(recon_batch, batch, mu, logvar, BIN_MASK, CONT_MASK)

        # Accumulate the total loss
        total_loss += loss.item()

# Compute the average loss over the entire validation dataset
average_loss = total_loss / len(val_loader.dataset)
print(f"Validation Loss: {average_loss:.4f}")

"""Testing and Validations
 - Evaluate Reccomendations
 - Fine Tune Model
"""


Validation Loss: 2.1485


'Testing and Validations\n - Evaluate Reccomendations\n - Fine Tune Model\n'